# EnsembleRetriever로 하이브리드 검색 구현

이 노트북에서는 LangChain의 `EnsembleRetriever`를 사용하여 여러 검색 알고리즘을 결합한 하이브리드 검색 시스템을 구현합니다.

## 학습 목표
- EnsembleRetriever의 개념과 작동 원리 이해
- BM25 (sparse retriever)와 FAISS (dense retriever) 결합
- Reciprocal Rank Fusion 알고리즘 활용
- 런타임에서 가중치 동적 조정

## 목차
1. [EnsembleRetriever 개요](#overview)
2. [환경 설정](#setup)
3. [기본 구현](#basic-implementation)
4. [검색 결과 비교](#comparison)
5. [런타임 Config 변경](#runtime-config)
6. [요약](#summary)

## 1. EnsembleRetriever 개요 <a id='overview'></a>

`EnsembleRetriever`는 여러 검색기를 결합하여 더 강력한 검색 결과를 제공하는 LangChain의 기능입니다. 이 검색기는 다양한 검색 알고리즘의 장점을 활용하여 단일 알고리즘보다 더 나은 성능을 달성할 수 있습니다.

### 주요 특징
1. **여러 검색기 통합**: 다양한 유형의 검색기를 입력으로 받아 결과를 결합합니다.
2. **결과 재순위화**: [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) 알고리즘을 사용하여 결과의 순위를 조정합니다.
3. **하이브리드 검색**: 주로 `sparse retriever`(예: BM25)와 `dense retriever`(예: 임베딩 유사도)를 결합하여 사용합니다.

### 장점
- **Sparse retriever (BM25)**: 키워드 기반 검색에 효과적
- **Dense retriever (FAISS)**: 의미적 유사성 기반 검색에 효과적

이러한 상호 보완적인 특성으로 인해 `EnsembleRetriever`는 다양한 검색 시나리오에서 향상된 성능을 제공할 수 있습니다.

자세한 내용은 [LangChain 공식 문서](https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble)를 참조하세요.

## 2. 환경 설정 <a id='setup'></a>

필요한 라이브러리를 설치하고 API 키를 설정합니다.

In [1]:
# 필요한 패키지 설치
# !pip install langchain langchain-openai faiss-cpu python-dotenv rank-bm25

In [2]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

## 3. 기본 구현 <a id='basic-implementation'></a>

BM25 retriever와 FAISS retriever를 생성하고, 이를 EnsembleRetriever로 결합합니다.

In [3]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# 샘플 문서 리스트
doc_list = [
    "I like apples",
    "I like apple company",
    "I like apple's iphone",
    "Apple is my favorite company",
    "I like apple's ipad",
    "I like apple's macbook",
]

# BM25 Retriever 초기화 (Sparse Retriever)
# BM25는 전통적인 정보 검색 알고리즘으로, 단어의 빈도와 문서 내 중요도를 기반으로 검색합니다.
bm25_retriever = BM25Retriever.from_texts(
    doc_list,
)
bm25_retriever.k = 1  # BM25Retriever의 검색 결과 개수를 1로 설정합니다.

# FAISS Retriever 초기화 (Dense Retriever)
# FAISS는 벡터 유사도 기반 검색으로, 의미적 유사성을 기반으로 검색합니다.
embedding = OpenAIEmbeddings()  # OpenAI 임베딩을 사용합니다.
faiss_vectorstore = FAISS.from_texts(
    doc_list,
    embedding,
)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 1})

# 앙상블 retriever를 초기화합니다.
# weights 매개변수로 각 retriever의 가중치를 설정할 수 있습니다.
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever],
    weights=[0.7, 0.3],  # BM25에 70%, FAISS에 30%의 가중치 부여
)

## 4. 검색 결과 비교 <a id='comparison'></a>

동일한 쿼리에 대해 각 retriever의 결과를 비교해봅니다.

### 테스트 1: "my favorite fruit is apple"

이 쿼리는 'favorite'와 'apple' 키워드를 포함하고 있어 BM25가 유리하며, 'fruit'과 'apple'의 의미적 연관성도 있어 FAISS도 관련 결과를 찾을 수 있습니다.

In [4]:
# 검색 결과 문서를 가져옵니다.
query = "my favorite fruit is apple"
ensemble_result = ensemble_retriever.invoke(query)
bm25_result = bm25_retriever.invoke(query)
faiss_result = faiss_retriever.invoke(query)

# 가져온 문서를 출력합니다.
print("[Ensemble Retriever]")
for doc in ensemble_result:
    print(f"Content: {doc.page_content}")
    print()

print("[BM25 Retriever]")
for doc in bm25_result:
    print(f"Content: {doc.page_content}")
    print()

print("[FAISS Retriever]")
for doc in faiss_result:
    print(f"Content: {doc.page_content}")
    print()

[Ensemble Retriever]
Content: Apple is my favorite company

Content: I like apples

[BM25 Retriever]
Content: Apple is my favorite company

[FAISS Retriever]
Content: I like apples



### 테스트 2: "Apple company makes my favorite iphone"

이 쿼리는 'Apple company', 'favorite', 'iphone' 등 여러 키워드를 포함하고 있어 각 retriever가 다른 관점에서 관련 문서를 찾을 수 있습니다.

In [5]:
# 검색 결과 문서를 가져옵니다.
query = "Apple company makes my favorite iphone"
ensemble_result = ensemble_retriever.invoke(query)
bm25_result = bm25_retriever.invoke(query)
faiss_result = faiss_retriever.invoke(query)

# 가져온 문서를 출력합니다.
print("[Ensemble Retriever]")
for doc in ensemble_result:
    print(f"Content: {doc.page_content}")
    print()

print("[BM25 Retriever]")
for doc in bm25_result:
    print(f"Content: {doc.page_content}")
    print()

print("[FAISS Retriever]")
for doc in faiss_result:
    print(f"Content: {doc.page_content}")
    print()

[Ensemble Retriever]
Content: Apple is my favorite company

Content: I like apple's iphone

[BM25 Retriever]
Content: Apple is my favorite company

[FAISS Retriever]
Content: I like apple's iphone



### 결과 분석

위의 결과에서 볼 수 있듯이:
- **BM25**: 키워드 매칭에 강점을 보입니다 (예: 'favorite', 'company' 등의 정확한 매칭)
- **FAISS**: 의미적 유사성에 강점을 보입니다 (예: 'fruit'과 'apple'의 연관성, 'iphone'과 관련된 문맥)
- **Ensemble**: 두 방법의 장점을 결합하여 더 포괄적인 결과를 제공합니다

## 5. 런타임 Config 변경 <a id='runtime-config'></a>

런타임에서도 retriever의 속성을 변경할 수 있습니다. 이는 `ConfigurableField` 클래스를 사용하여 가능합니다.

이 기능은 다양한 검색 시나리오에 맞춰 가중치를 동적으로 조정할 때 유용합니다.

In [6]:
from langchain_core.runnables import ConfigurableField

# ConfigurableField를 사용하여 런타임에 가중치를 변경할 수 있도록 설정합니다.
ensemble_retriever = EnsembleRetriever(
    # 리트리버 목록을 설정합니다. 여기서는 bm25_retriever와 faiss_retriever를 사용합니다.
    retrievers=[bm25_retriever, faiss_retriever],
).configurable_fields(
    weights=ConfigurableField(
        # 검색 매개변수의 고유 식별자를 설정합니다.
        id="ensemble_weights",
        # 검색 매개변수의 이름을 설정합니다.
        name="Ensemble Weights",
        # 검색 매개변수에 대한 설명을 작성합니다.
        description="Ensemble Weights for BM25 and FAISS retrievers",
    )
)

### BM25에 100% 가중치 부여

키워드 매칭이 중요한 경우 BM25에 모든 가중치를 부여할 수 있습니다.

In [7]:
# BM25에 100%, FAISS에 0% 가중치 부여
config = {"configurable": {"ensemble_weights": [1, 0]}}

# config 매개변수를 사용하여 검색 설정을 지정합니다.
docs = ensemble_retriever.invoke("my favorite fruit is apple", config=config)
docs  # 검색 결과인 docs를 출력합니다.

[Document(page_content='Apple is my favorite company'),
 Document(page_content='I like apples')]

### FAISS에 100% 가중치 부여

의미적 유사성이 중요한 경우 FAISS에 모든 가중치를 부여할 수 있습니다.

In [8]:
# BM25에 0%, FAISS에 100% 가중치 부여
config = {"configurable": {"ensemble_weights": [0, 1]}}

# config 매개변수를 사용하여 검색 설정을 지정합니다.
docs = ensemble_retriever.invoke("my favorite fruit is apple", config=config)
docs  # 검색 결과인 docs를 출력합니다.

[Document(page_content='I like apples'),
 Document(page_content='Apple is my favorite company')]

### 균형잡힌 가중치 설정

다양한 가중치 조합을 테스트하여 최적의 설정을 찾을 수 있습니다.

In [9]:
# 다양한 가중치 조합 테스트
weight_combinations = [
    [0.5, 0.5],  # 균등 분배
    [0.3, 0.7],  # FAISS 중심
    [0.7, 0.3],  # BM25 중심
]

query = "my favorite fruit is apple"

for weights in weight_combinations:
    config = {"configurable": {"ensemble_weights": weights}}
    docs = ensemble_retriever.invoke(query, config=config)
    
    print(f"Weights {weights}:")
    for i, doc in enumerate(docs, 1):
        print(f"{i}. {doc.page_content}")
    print()

Weights [0.5, 0.5]:
1. I like apples
2. Apple is my favorite company

Weights [0.3, 0.7]:
1. I like apples
2. Apple is my favorite company

Weights [0.7, 0.3]:
1. Apple is my favorite company
2. I like apples



## 6. 요약 <a id='summary'></a>

이 노트북에서는 LangChain의 `EnsembleRetriever`를 사용하여 하이브리드 검색 시스템을 구현했습니다.

### 핵심 포인트

1. **EnsembleRetriever의 장점**
   - 여러 검색 알고리즘의 장점을 결합
   - Reciprocal Rank Fusion을 통한 효과적인 결과 통합
   - 유연한 가중치 조정

2. **BM25 vs FAISS**
   - BM25: 정확한 키워드 매칭에 효과적
   - FAISS: 의미적 유사성 검색에 효과적
   - 두 방법의 조합으로 더 나은 검색 결과 달성

3. **동적 설정**
   - ConfigurableField를 통한 런타임 가중치 조정
   - 다양한 검색 시나리오에 맞춘 최적화 가능

### 활용 시나리오

- **문서 검색 시스템**: 사용자 쿼리의 특성에 따라 가중치 조정
- **FAQ 시스템**: 키워드와 의미적 검색을 모두 활용
- **추천 시스템**: 다양한 관점에서의 유사 아이템 검색

### 참고 자료

- [LangChain EnsembleRetriever 문서](https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble)
- [Reciprocal Rank Fusion 논문](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf)
- [BM25 알고리즘 설명](https://en.wikipedia.org/wiki/Okapi_BM25)
- [FAISS 라이브러리](https://github.com/facebookresearch/faiss)